# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Нужно построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Построим модель с максимально большим значением *accuracy*. Доведём долю правильных ответов по крайней мере до 0.75.

## Импорт необходимых библиотек

In [1]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Откроем и изучим файл

В этом пункте:

1. Откроем файл;
2. Сравним данные в файле с описанием в документации;
3. Посмотрим на числовые характеристики;

### Откроем файл

Путь к файлу: `/datasets/users_behavior.csv`. Воспользуемся методом `read_csv()` библиотеки `pandas`.

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


### Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В файле информация о 3214 пользователях. В качестве параметров будем использовать информацию об использовании сервисов тарифа.

4 параметра об использовании сервисов представлены вещественным типом данных `float64`.
1 параметр, описывающий тариф клиента, представлен целочисленным типом `int64`.

Пропусков в данных нет.

### Числовые характеристики

Воспользуемся методом `describe()`.

In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Минимальные значения по звонкам, минутам, сообщениям и интернет-трафику равны 0. Это говорит о том, что клиент не пользовался услугой.

## Разделим данные на выборки

По описанию проекта нам нужно построить систему, предлагающую клиентам тариф. Значит в качестве целевого признака будем использовать данные столбца `is_ultra`.

Информация о тарифе является категориальным набором данных. По этой причине вся задача пстроения системы, по которой будет определяться тариф, сводится к **классификации**.

Для построения модели нам необходимы обучающая, валидационная и тестовая выборки. Отдельно предоставленной тестовой выборки нет, поэтому разделим исходные данные в соотношении 3:1:1, где 3/5 будет обучающая и по 1/5 валидационная и тестовая.

Для разделения воспользуемся функцией `train_test_split`. Она разделяет датасет на обучающую и тестовую выборки, но можно получить и валидационную вместо тестовой. Последовательно отделим от датасета по 1/5. Так мы получим сначала тестовую, потом валидационную.

В качестве параметров функции `train_test_split` укажем датасет, который будем разделять, размер отделяемой выборки `test_size`, зафиксируем псевдослучайность параметром `random_state`, отличным от значения `None`.

In [5]:
# Обучающая (в соотношении 4/5) и тестовая выборки
df_train, df_test = train_test_split(df, test_size=0.2, random_state=12345)

# Обучающая (в соотношении 3/5) и валидационная выборки
df_train, df_valid = train_test_split(df_train, test_size=0.25, random_state=12345)

Посмотрим на размеры полученных выборок:

In [6]:
print(f"Обучающая выборка: объектов - {df_train.shape[0]}, признаков - {df_train.shape[1]} ")
print(f"Тестовая выборка: объектов {df_test.shape[0]}, признаков - {df_test.shape[1]}")
print(f"Валидационная выборка: объектов {df_valid.shape[0]}, признаков - {df_valid.shape[1]}")

Обучающая выборка: объектов - 1928, признаков - 5 
Тестовая выборка: объектов 643, признаков - 5
Валидационная выборка: объектов 643, признаков - 5


**Вывод**

Таким образом мы получили необходиые наборы данных для построения и обучения модели, разделив исходные данные на 3 части в соотношении 3:1:1.

## Исследуем модели

В этом пункте мы рассмотрим 3 варианта моделей:
1. Дерево решений;
2. Случайный лес;
3. Логистическую регрессию.

Выбирать лучшее будем по показателю **качества** `accuracy`, так как это самый важный критерий для бизнеса: чем выше качество, тем больше прибыли. По результатам выберем наиболее подходящую для задачи подбора тарифа.

### Дерево решений

Напишем функцию, которая поможет нам настроить гиперпараметры дерева решений.

Самым важным гиперпараметром является максимальная глубина `max_depth`. Переберем разные значения `max_depth` и сравним качество моделей в разных вариантах, чтобы выбрать наилучший.

Обучать модель будем на тренировочном датасете, а проверять точность `accuracy` на валидационной.

In [7]:
def decision_tree_modeling(train, valid):   
    target_train = train['is_ultra']
    features_train = train.drop(['is_ultra'], axis=1)
    
    target_valid = valid['is_ultra']
    features_valid = valid.drop(['is_ultra'], axis=1)
    
    depths = []
    results = []
    
    for depth in range(1, 10):
        model = DecisionTreeClassifier(max_depth=depth, random_state=12345) # получение структуры Дерева решений
        model.fit(features_train, target_train) # обучение модели на тренировочной выборке
        
        predictions_valid = model.predict(features_valid) # предсказания модели на валидационной выборке
        
        depths.append(depth)
        results.append(accuracy_score(target_valid, predictions_valid))

    return pd.DataFrame({'depth': depths, 'accuracy': results})

In [8]:
dec_tree_df = decision_tree_modeling(df_train, df_valid)
dec_tree_df

,depth,accuracy
0,1,0.738725
1,2,0.757387
2,3,0.765163
3,4,0.763608
4,5,0.758942
5,6,0.757387
6,7,0.774495
7,8,0.766719
8,9,0.762053


Найдем максимальное значение `accuracy` и соответствующую ей глубину `depth`:

In [9]:
dec_tree_df[dec_tree_df['accuracy'] == dec_tree_df['accuracy'].max()]

,depth,accuracy
6,7,0.774495


**Вывод**

По результатам мы видим, что максимальное значение качества `accuracy = 0.774495` достигается при значении `max_depth = 7`. Меньше этого значения - модель недообучается. Если же больше - начинает переобучаться.

### Случайный лес

Для модели случайного леса важным гиперпараметром является число деревьев `n_estimators`.

In [10]:
def random_forest_modeling(train, valid):
    target_train = train['is_ultra']
    features_train = train.drop(['is_ultra'], axis=1)
    
    target_valid = valid['is_ultra']
    features_valid = valid.drop(['is_ultra'], axis=1)
    
    for est in range(1, 25):
        model = RandomForestClassifier(n_estimators=est, random_state=12345) # получение структуры Случайного леса
        model.fit(features_train, target_train) # обучение модели на тренировочной выборке
        
        predictions_valid = model.predict(features_valid) # предсказания модели на валидационной выборке
    
        print('est =', est, ': ', end='') # вывод значения гиперпараметра
        print(accuracy_score(target_valid, predictions_valid)) # вывод соответствующего гиперпараметру значения качества

In [11]:
random_forest_modeling(df_train, df_valid)

est = 1 : 0.702954898911353
est = 2 : 0.7573872472783826
est = 3 : 0.744945567651633
est = 4 : 0.7651632970451011
est = 5 : 0.7620528771384136
est = 6 : 0.7698289269051322
est = 7 : 0.7713841368584758
est = 8 : 0.7869362363919129
est = 9 : 0.7838258164852255
est = 10 : 0.7884914463452566
est = 11 : 0.7807153965785381
est = 12 : 0.7822706065318819
est = 13 : 0.7776049766718507
est = 14 : 0.7853810264385692
est = 15 : 0.7838258164852255
est = 16 : 0.7838258164852255
est = 17 : 0.7776049766718507
est = 18 : 0.7869362363919129
est = 19 : 0.7869362363919129
est = 20 : 0.7900466562986003
est = 21 : 0.7884914463452566
est = 22 : 0.7931570762052877
est = 23 : 0.7884914463452566
est = 24 : 0.7900466562986003


**Вывод**

По результатам мы видим, что максимальное значение качества `accuracy` достигается при значении `n_estimators` равном 13. Меньше этого значения - модель недообучается. Если же больше - наблюдаются повторения. При этом, если увеличивать количество деревьев, то модель будет медленнее обучаться.

### Логистическая регрессия

У этой модели мало гиперпараметров, поэтому вероятность её переобучения мала. Укажем гиперпараметр `solver` равным `liblinear`, как лучший и быстрый вариант для небольших датасетов.

In [12]:
def logistic_regression_modeling(train, valid):
    target_train = train['is_ultra']
    features_train = train.drop(['is_ultra'], axis=1)
    
    target_valid = valid['is_ultra']
    features_valid = valid.drop(['is_ultra'], axis=1)
    
    model = LogisticRegression(random_state=12345, solver='liblinear') # инициализирование модели логистической регрессии 
    model.fit(features_train, target_train) # обучение модели на тренировочной выборке
    result = model.score(features_valid, target_valid) # получите метрику качества модели на валидационной выборке
    predictions_valid = model.predict(features_valid) # предсказания модели на валидационной выборке

    print(accuracy_score(target_valid, predictions_valid)) # вывод соответствующего гиперпараметру значения качества

In [13]:
logistic_regression_modeling(df_train, df_valid)

0.6967340590979783


**Вывод**

По результатам мы видим, что значение качества `accuracy` не удовлетворяет изначальному условию "не менее 0.75".

### Вывод

Таким образом, лучшим выбором по показателю качества является Случайный лес. На втором месте Дерево решений. Однако, если принять во внимание скорость работы модели, то быстрее будет Дерево решений.

При разнице наилучших показателей качества на значение порядка 0.01 лучшим вариантом модели будет Дерево решений.

## Проверим модель на тестовой выборке

В этом пункте мы:
1. проверим выбранную модель на тестовой выборке;
2. посмотрим на количество ошибок между предсказаниями и правильными ответами.

Разделим обучающую и тестовую выборки на целевой признак и остальные для каждой

In [14]:
# обучающая
target_train = df_train['is_ultra']
features_train = df_train.drop(['is_ultra'], axis=1)

# тестовая
target_test = df_test['is_ultra'].reset_index(drop=True)
features_test = df_test.drop(['is_ultra'], axis=1).reset_index(drop=True)

Инициализируем модель с лучшим значением гиперпараметра `max_depth`, равным 7.

In [15]:
model = DecisionTreeClassifier(random_state=12345, max_depth=7) # инициализация модели
model.fit(features_train, target_train); # обучение модели

Напишем функцию для тестирования. В ней мы определим количество ошибок и посчитаем показатель качества, соответствующего этому количеству.

In [16]:
def testing_decision_tree_model(features_test, answers):
    predictions = model.predict(features_test) # предсказания модели на тестовой выборке
    
    # подсчет количества ошибок
    errors = 0
    for i in range(len(answers)):
        if answers[i] != predictions[i]:
            errors += 1
    
    
    print("Ошибок:", errors) # Вывод количества ошибок
    print("Accuracy:", (len(predictions) - errors) / len(predictions)) # Подсчет и вывод accuracy

In [17]:
testing_decision_tree_model(features_test, target_test)

Ошибок: 136
Accuracy: 0.7884914463452566


### Вывод

Выбранная нами модель Дерева решений хорошо показала себя на тестовой выборке. Показатель качества оказался равным 0.79, что выше, чем на валидационной.

## Чек-лист готовности проекта

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
